In [52]:
# !pip install transformers==4.57.3
# !pip install qwen-tts==0.1.1
# !pip install peft==0.10.0
# !pip install datasets librosa accelerate

In [53]:
# !git clone https://github.com/QwenLM/Qwen3-TTS.git

In [ ]:
# %cd Qwen3-TTS

In [55]:
# !pip install -e .

In [56]:
# !pip install -U flash-attn

In [ ]:
# import os
# import json
# import torch
# import torchaudio
# from tqdm import tqdm
# from torch.utils.data import Dataset, DataLoader

# from transformers import AutoConfig
# from peft import LoraConfig, get_peft_model

# from qwen_tts.core.models.processing_qwen3_tts import Qwen3TTSProcessor
# from qwen_tts.core.models.modeling_qwen3_tts import Qwen3TTSForConditionalGeneration

# # =============================
# # CONFIG
# # =============================

# OUTPUT_DIR = "output"
# MODEL_NAME = "Qwen/Qwen3-TTS-12Hz-0.6B-Base"
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# SAMPLE_RATE = 24000

# EPOCHS = 3
# BATCH_SIZE = 2
# LR = 2e-4

# # =============================
# # DATASET
# # =============================

# class TTSDataset(Dataset):
#     def __init__(self, manifest_path, processor):
#         self.processor = processor

#         with open(manifest_path, "r", encoding="utf-8") as f:
#             self.data = [json.loads(line) for line in f if line.strip()]

#         speakers = sorted(set(d["speaker"] for d in self.data))
#         self.spk2id = {s: i for i, s in enumerate(speakers)}
#         print("Speaker mapping:", self.spk2id)

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         item = self.data[idx]

#         wav, sr = torchaudio.load(item["audio"])
#         wav = wav.mean(0)  # mono

#         if sr != SAMPLE_RATE:
#             wav = torchaudio.functional.resample(wav, sr, SAMPLE_RATE)

#         speaker_id = self.spk2id[item["speaker"]]

#         inputs = self.processor(
#             text=item["text"],
#             audio=wav,
#             sampling_rate=SAMPLE_RATE,
#             speaker_id=speaker_id,
#             return_tensors="pt"
#         )

#         return {
#             "input_ids": inputs["input_ids"].squeeze(0),
#             "labels": inputs["audio_values"].squeeze(0),  # 🔥 ใช้ audio_values
#         }


# def collate_fn(batch):
#     input_ids = torch.nn.utils.rnn.pad_sequence(
#         [b["input_ids"] for b in batch],
#         batch_first=True,
#         padding_value=0
#     )

#     labels = torch.nn.utils.rnn.pad_sequence(
#         [b["labels"] for b in batch],
#         batch_first=True,
#         padding_value=0.0  # 🔥 audio ใช้ 0 ไม่ใช่ -100
#     )

#     return {
#         "input_ids": input_ids,
#         "labels": labels
#     }


# # =============================
# # TRAIN
# # =============================

# def train(manifest_path, output_dir):

#     print("Loading processor...")
#     processor = Qwen3TTSProcessor.from_pretrained(
#         MODEL_NAME,
#         trust_remote_code=True
#     )

#     print("Loading model...")
#     model = Qwen3TTSForConditionalGeneration.from_pretrained(
#         MODEL_NAME,
#         torch_dtype=torch.bfloat16,
#         trust_remote_code=True
#     ).to(DEVICE)

#     # LoRA
#     lora_config = LoraConfig(
#         r=32,
#         lora_alpha=64,
#         target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
#         lora_dropout=0.05,
#         bias="none",
#         task_type="CAUSAL_LM"
#     )

#     model = get_peft_model(model, lora_config)
#     model.print_trainable_parameters()

#     dataset = TTSDataset(manifest_path, processor)

#     dataloader = DataLoader(
#         dataset,
#         batch_size=BATCH_SIZE,
#         shuffle=True,
#         collate_fn=collate_fn
#     )

#     optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

#     model.train()

#     for epoch in range(EPOCHS):
#         total_loss = 0

#         for batch in tqdm(dataloader):
#             batch = {k: v.to(DEVICE) for k, v in batch.items()}

#             outputs = model(
#                 input_ids=batch["input_ids"],
#                 labels=batch["labels"]
#             )

#             loss = outputs.loss

#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item()

#         print(f"[{output_dir}] Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")

#     os.makedirs(output_dir, exist_ok=True)
#     model.save_pretrained(output_dir)
#     processor.save_pretrained(output_dir)

#     print(f"Training complete → saved to {output_dir}")


# # =============================
# # GENERATE (เลือก speaker)
# # =============================

# def generate(text, speaker_name):

#     processor = Qwen3TTSProcessor.from_pretrained(
#         OUTPUT_DIR,
#         trust_remote_code=True
#     )

#     model = Qwen3TTSForConditionalGeneration.from_pretrained(
#         OUTPUT_DIR,
#         torch_dtype=torch.bfloat16,
#         trust_remote_code=True
#     ).to(DEVICE)

#     model.eval()

#     inputs = processor(text=text, return_tensors="pt")
#     input_ids = inputs["input_ids"].to(DEVICE)

#     codes, _ = model.generate(
#         input_ids=input_ids,   # 🔥 ไม่ต้องใส่ []
#         languages=["auto"],
#         speakers=[speaker_name],
#     )

#     wav = processor.decode(codes[0])

#     torchaudio.save(
#         f"output_{speaker_name}.wav",
#         wav.unsqueeze(0).cpu(),
#         SAMPLE_RATE
#     )

#     print("Saved:", f"output_{speaker_name}.wav")

In [58]:
# import json

# base_path = "C:/Users/User/Comvi/Voice/"

# def json_to_jsonl(input_path, output_path):
#     with open(base_path+input_path, "r", encoding="utf-8") as f:
#         data = json.load(f)

#     for data_dict in data:
#         data_dict["audio"] = base_path + data_dict["audio"]

#     with open(base_path+output_path, "w", encoding="utf-8") as f:
#         for item in data:
#             f.write(json.dumps(item, ensure_ascii=False) + "\n")

#     print("✅ Converted to JSONL:", base_path+output_path)


# json_to_jsonl("manifest_whisper_text.json", "manifest_whisper_text.jsonl")
# json_to_jsonl("manifest_pathumma_text.json", "manifest_pathumma_text.jsonl")

In [7]:
import json

def convert_manifest_for_qwen3(
    input_manifest,
    output_manifest,
    ref_audio_path
):
    """
    แปลง manifest เดิม -> format ที่ Qwen3-TTS official ต้องการ
    
    Parameters:
    - input_manifest: path ไฟล์ jsonl เดิม
    - output_manifest: path ไฟล์ jsonl ใหม่
    - ref_audio_path: path ของไฟล์ reference เสียง A (ใช้ไฟล์เดียวทั้ง dataset)
    """

    count = 0

    with open(input_manifest, "r", encoding="utf-8") as fin, \
         open(output_manifest, "w", encoding="utf-8") as fout:

        for line in fin:
            if not line.strip():
                continue

            item = json.loads(line)

            text = f"[{item['speaker']}] " + item["text"]

            new_item = {
                "audio": item["audio"],
                "text": text,
                "ref_audio": ref_audio_path[item['speaker']]
            }

            fout.write(json.dumps(new_item, ensure_ascii=False) + "\n")
            count += 1

    print(f"✅ Converted {count} samples")
    print(f"Saved to: {output_manifest}")

ref_dict = {"Lisa": "C:/Users/User/Comvi/Voice/Lisa/dataset_denoised/gEMrqw-pAy4/SPEAKER_01_sent_0001_DeepFilterNet3.wav",
                    "Bambam": "C:/Users/User/Comvi/Voice/Bambam/dataset_denoised/zbo7Mk3ryaI/SPEAKER_00_sent_0001_DeepFilterNet3.wav",
                    "IU": "C:/Users/User/Comvi/Voice/IU/dataset_denoised/wCbUWU4l_Ko/SPEAKER_01_sent_0000_DeepFilterNet3.wav",
                    "IVE": "C:/Users/User/Comvi/Voice/IVE/dataset_denoised/_037bSnAyRg/SPEAKER_01_sent_0007_DeepFilterNet3.wav"
                    }

convert_manifest_for_qwen3(
    input_manifest="C:/Users/User/Comvi/Voice/manifest_whisper_text.jsonl",
    output_manifest="C:/Users/User/Comvi/Voice/manifest_whisper_train_raw.jsonl",
    ref_audio_path=ref_dict
)

convert_manifest_for_qwen3(
    input_manifest="C:/Users/User/Comvi/Voice/manifest_pathumma_text.jsonl",
    output_manifest="C:/Users/User/Comvi/Voice/manifest_pathumma_train_raw.jsonl",
    ref_audio_path=ref_dict
)


✅ Converted 2558 samples
Saved to: C:/Users/User/Comvi/Voice/manifest_whisper_train_raw.jsonl
✅ Converted 2590 samples
Saved to: C:/Users/User/Comvi/Voice/manifest_pathumma_train_raw.jsonl


In [ ]:
# train("C:/Users/User/Comvi/Voice/manifest_whisper_train_raw.jsonl", "C:/Users/User/Comvi/Voice/research_tts_whisper")
# print("\n\n")
# train("C:/Users/User/Comvi/Voice/manifest_pathumma_train_raw.jsonl", "C:/Users/User/Comvi/Voice/research_tts_pathumma")

In [2]:
%cd Qwen3-TTS/finetuning


c:\Users\User\Comvi\Voice\Qwen3-TTS\finetuning


In [19]:
import os
import librosa
import soundfile as sf

def convert_to_24k(input_path, output_path=None):
    """
    Convert audio file to 24kHz mono WAV.

    Args:
        input_path (str): path to original audio file
        output_path (str, optional): output file path.
                                     If None, will overwrite with _24k.wav suffix

    Returns:
        str: path to converted file
    """

    # Load audio (auto-detect original sample rate)
    audio, sr = librosa.load(input_path, sr=None, mono=True)

    # Resample if needed
    if sr != 24000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=24000)
        sr = 24000

    # Set output path
    if output_path is None:
        base, ext = os.path.splitext(input_path)
        output_path = base + "_24k.wav"

    # Save as WAV 24kHz
    sf.write(output_path, audio, sr)

    return output_path

folder_list = ["C:/Users/User/Comvi/Voice/Lisa", "C:/Users/User/Comvi/Voice/Bambam", "C:/Users/User/Comvi/Voice/IU", "C:/Users/User/Comvi/Voice/IVE"]
file_path_list = []

for folder in folder_list:
    sub_folder = folder + "/dataset"
    for file in os.listdir(sub_folder):
        folder_path = folder + "/dataset/" + file
        for wav_file in os.listdir(folder_path):
            if wav_file.endswith(".wav"):
                file_path = folder_path + "/" + wav_file
                file_path_list.append(file_path)

for file_path in file_path_list:
    convert_to_24k(file_path, file_path)


In [28]:
import json
import soundfile as sf

manifest_path = "C:/Users/User/Comvi/Voice/manifest_whisper_train_raw.jsonl"

with open(manifest_path, "r", encoding="utf-8") as f:
    for line in f:
        data_dict = json.loads(line.strip())
        wav_path = data_dict["audio"]

        wav, sr = sf.read(wav_path)

        if sr != 24000:
            print("NOT 24K:", wav_path, "->", sr)

manifest_path = "C:/Users/User/Comvi/Voice/manifest_pathumma_train_raw.jsonl"

with open(manifest_path, "r", encoding="utf-8") as f:
    for line in f:
        data_dict = json.loads(line.strip())
        wav_path = data_dict["audio"]

        wav, sr = sf.read(wav_path)

        if sr != 24000:
            print("NOT 24K:", wav_path, "->", sr)

In [20]:
!python prepare_data.py --device cuda:0 --tokenizer_model_path Qwen/Qwen3-TTS-Tokenizer-12Hz --input_jsonl "C:/Users/User/Comvi/Voice/manifest_whisper_train_raw.jsonl" --output_jsonl "C:/Users/User/Comvi/Voice/manifest_whisper_train_with_code.jsonl"


********
********
 


'sox' is not recognized as an internal or external command,
operable program or batch file.
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [21]:
!python prepare_data.py --device cuda:0 --tokenizer_model_path Qwen/Qwen3-TTS-Tokenizer-12Hz --input_jsonl "C:/Users/User/Comvi/Voice/manifest_pathumma_train_raw.jsonl" --output_jsonl "C:/Users/User/Comvi/Voice/manifest_pathumma_train_with_code.jsonl"


********
********
 


'sox' is not recognized as an internal or external command,
operable program or batch file.
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [3]:
!python sft_12hz.py --init_model_path "C:/Users/User/.cache/huggingface/hub/models--Qwen--Qwen3-TTS-12Hz-1.7B-Base/snapshots/fd4b254389122332181a7c3db7f27e918eec64e3" --output_model_path "C:/Users/User/Comvi/Voice/output_whisper" --train_jsonl "C:/Users/User/Comvi/Voice/manifest_whisper_train_with_code.jsonl" --batch_size 1 --lr 1e-6 --num_epochs 10 --speaker_name multi_speaker

^C


In [ ]:
!python sft_12hz.py --init_model_path "C:/Users/User/.cache/huggingface/hub/models--Qwen--Qwen3-TTS-12Hz-1.7B-Base/snapshots/fd4b254389122332181a7c3db7f27e918eec64e3" --output_model_path "C:/Users/User/Comvi/Voice/output_pathumma" --train_jsonl "C:/Users/User/Comvi/Voice/manifest_pathumma_train_with_code.jsonl" --batch_size 1 --lr 1e-6 --num_epochs 10 --speaker_name multi_speaker

^C


In [ ]:
import torch
import soundfile as sf
from qwen_tts import Qwen3TTSModel

device = "cuda:0"

tts = Qwen3TTSModel.from_pretrained(
    "output/checkpoint-epoch-4",
    device_map=device,
    dtype=torch.float16,
)

ref_dict = {"Lisa": "C:/Users/User/Comvi/Voice/Lisa/dataset_denoised/gEMrqw-pAy4/SPEAKER_01_sent_0001_DeepFilterNet3.wav",
            "Bambam": "C:/Users/User/Comvi/Voice/Bambam/dataset_denoised/zbo7Mk3ryaI/SPEAKER_00_sent_0001_DeepFilterNet3.wav",
            "IU": "C:/Users/User/Comvi/Voice/IU/dataset_denoised/wCbUWU4l_Ko/SPEAKER_01_sent_0000_DeepFilterNet3.wav",
            "IVE": "C:/Users/User/Comvi/Voice/IVE/dataset_denoised/_037bSnAyRg/SPEAKER_01_sent_0007_DeepFilterNet3.wav"
            }

def generate_voice(speaker_name, text_input, output_file):
    ref_audio_path = ref_dict[speaker_name]
    text_input = f"[{speaker_name}] " + text_input

    wavs, sr = tts.generate_custom_voice(
        text=text_input,
        ref_audio=ref_audio_path,
    )

    sf.write(output_file, wavs[0], sr)

song1 = """ระหว่างที่มีดาวดวงนึงโคจรในอวกาศ..
จากวันนั้นฉันก็มองโลกไม่เหมือนเดิม...
สิ่งที่สวยงามในชีวิตพบเจอ..
คงไม่เหลือคุณค่าอีกแล้ว.. ถูกมาแทนด้วยดวงตาคู่นั้นของ..เธอ...
ปฏิทินจากนี้มีไว้เพื่อนับวัน.. นาฬิกามีไว้เฝ้าคอยนับนาที...
ของชีวิตฉันนับต่อจากนี้ ที่จะมีไว้เพื่อเธอเท่านั้น...

ขออยู่ในชีวิตที่เหลือของเธอได้ไหม..
อยากลืมตาแล้วได้พบเธอจนวันสุดท้าย..
อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน
และเอนไปจุมพิตเธอซักล้านครั้ง
อยู่กับฉันไปนานๆ... นะเธอ.....

ยังมีเพลงรักเป็นพันบทเพลงรอแชร์ให้เธอได้ฟัง..
ยังมีอีกหลายทริปที่เรานั้นยังต้องไปเที่ยวด้วยกัน...
ไม่รวมอีกเรื่องที่ยังสำคัญ ลูกเราที่สักวันจะเกิดมา
ด้วยพันธสัญญาความรักของเรา...

และมีดาวหางดวงนึงที่ยังโคจรในอวกาศ..
ในช่วงชีวิตจะมีหนึ่งครั้งที่มองเห็นได้ด้วยตา..
ดาวที่ฉันยังไม่เคยเห็นมาก่อน
ขอให้ถึงวันนั้น
ได้มีเธอรอดูมันด้วยกันกับฉัน...

ขออยู่ในชีวิตที่เหลือของเธอได้ไหม..
อยากลืมตาแล้วได้พบเธอจนวันสุดท้าย..
อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน
และเอนไปจุมพิตเธอซักล้านครั้ง
อยู่กับฉันไปนานๆ... นะเธอ.....


ขออยู่ในชีวิตที่เหลือของเธอได้ไหม..
อยากลืมตาแล้วได้พบเธอจนวันสุดท้าย..
อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน
และเอนไปจุมพิตเธอซักล้านครั้ง
อยู่กับฉันไปนานๆ... นะเธอ.....

อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน
และเอนไปจุมพิตเธอซักล้านครั้ง.....
อยู่กับฉันไปนานๆ... นะเธอ....."""

song2 = """
ได้มาพ้อ..น้องแก้มอ่องต่องในวันสงกรานต์..
เต้นอยู่หน้าฮ้าน.. เมื่อคืนเทศกาลวันไหล...
เจ้ายิ้มเข้ามาหยอก บอกว่าฮักอ้ายเบิดใจ..
เข้ามาซบที่ตรงไหล่.. เจ้าว่าหัวใจเฮาตรงกัน

อยู่กันสองคืน.. เจ้ามาขอกลับไปเฮ็ดงาน..
ก่อนกระเจียวบาน.. บ่ต้องย่านสิกลับมาหา..
อ้ายก็ถ่าจนเศร้า.. กอดรูปเจ้าทั้งน้ำตา..
เจ้าคงลืมสัญญา..สามเดือนกว่ากว่าแล้วไปอยู่ไส....

อยาก...เจอ ได้แค่เพียงคิดฮอด รอวันได้กอด...เธอ
เฝ้าแต่ฝันละเมอกับคำว่าฮักที่เธอนั้นบอก
อ้ายบ่ฮู้ว่าจริงหรือหยอก
ย่านคนสวยน้องสิมาหลอกให้อ้ายช้ำ..

ดอกกระเจียวบาน.. อีกไม่นานก็คงสิเฉา..
อ้ายก็รอเจ้าอยู่คือเก่า ไปเป็นผู้สาวผู้ใดหนอ..นาง
นั่ง...คึดฮอด..บ่ได้นอนจนฟ้าสาง..
ย่านความฮักเฮาแตกม่าง
เจ้าลืมทุกอย่างของสองเฮา...

ดอกกระเจียวบาน.. ผ่านหน้าแล้งเจ้าไปอยู่ไส..
เข้าหน้าฝนบ่โดนเท่าไหร่..
น้ำตาของอ้ายกะไหลหย่าว...
ใจสวอย..อย่าให้คอยถึงหน้าหนาว....
จนดอกกระเจียวของอ้ายเหี่ยวเฉา
เจ้ายังบ่..คืนมา.....

อยาก...เจอ ได้แค่เพียงคิดฮอด รอวันได้กอด...เธอ
เฝ้าแต่ฝันละเมอกับคำว่าฮักที่เธอนั้นบอก
อ้ายบ่ฮู้ว่าจริงหรือหยอก
ย่านคนสวยน้องสิมาหลอกให้อ้ายช้ำ....

ดอกกระเจียวบาน.. อีกไม่นานก็คงสิเฉา..
อ้ายก็รอเจ้าอยู่คือเก่า ไปเป็นผู้สาวผู้ใดหนอ..นาง
นั่ง...คึดฮอด..บ่ได้นอนจนฟ้าสาง..
ย่านความฮักเฮาแตกม่าง
เจ้าลืมทุกอย่างของสองเฮา...

ดอกกระเจียวบาน.. ผ่านหน้าแล้งเจ้าไปอยู่ไส..
เข้าหน้าฝนบ่โดนเท่าไหร่..
น้ำตาของอ้ายกะไหลหย่าว...
ใจสวอย..อย่าให้คอยถึงหน้า....หนาว....
จนดอกกระเจียวของอ้ายเหี่ยวเฉา
เจ้ายังบ่..คืนมา........


ดอกกระเจียวบาน.. ผ่านหน้าแล้งเจ้าไปอยู่ไส..
เข้าหน้าฝนบ่โดนเท่าไหร่..
น้ำตาของอ้ายกะไหลหย่าว...
ใจสวอย..อย่าให้คอยถึงหน้า....หนาว....
จนดอกกระเจียวของอ้ายเหี่ยวเฉา
เจ้ายังบ่..คืนมา........

จนกระเจียวของอ้ายเหี่ยวเฉา
เจ้ายังบ่คืนมา...."""

generate_voice("Lisa", song1, "C:/Users/User/Comvi/Voice/Lisa_song1.wav")
generate_voice("Lisa", song2, "C:/Users/User/Comvi/Voice/Lisa_song2.wav")

generate_voice("Bambam", song1, "C:/Users/User/Comvi/Voice/Bambam_song1.wav")
generate_voice("Bambam", song2, "C:/Users/User/Comvi/Voice/Bambam_song2.wav")

generate_voice("IU", song1, "C:/Users/User/Comvi/Voice/IU_song1.wav")
generate_voice("IU", song2, "C:/Users/User/Comvi/Voice/IU_song2.wav")

generate_voice("IVE", song1, "C:/Users/User/Comvi/Voice/IVE_song1.wav")
generate_voice("IVE", song2, "C:/Users/User/Comvi/Voice/IVE_song2.wav")